In [1]:
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
# from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import calinski_harabasz_score
import random

import seaborn as sns
from app.src.api_osmr import get_matrix, parse_coordinates
from app.src.routing import optimize_routes, plot_route

import geopandas as gpd

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from typing import List, Optional, Tuple
import osmnx as ox
import networkx as nx
import folium
from folium import plugins
import numpy as np

random.seed(42)




load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\ortools.dll...


In [2]:
from loguru import logger

In [6]:
import pandas as pd
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
# from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import calinski_harabasz_score
import random

from app.src.api_osmr import get_matrix, parse_coordinates
from app.src.routing import optimize_routes, plot_route

from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import calinski_harabasz_score

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from typing import List

from abc import ABC, abstractmethod
from sklearn.cluster import MiniBatchKMeans
from scipy.spatial.distance import cdist
import numpy as np
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from typing import List, Optional, Tuple
import osmnx as ox
import networkx as nx
import folium
from folium import plugins
import numpy as np

equipaments = pd.read_csv("data/equipaments_model.csv")
equipaments.head()


random.seed(42)


class AbstractBaseClass(ABC):

    @abstractmethod
    def __init__():
        pass

    @abstractmethod
    def fit():
        pass

    @abstractmethod
    def predict():
        pass


from __future__ import print_function
import logging
from app.model.abstract_base_class import AbstractBaseClass
import scipy.cluster.hierarchy as sch
from sklearn.metrics import calinski_harabasz_score
from app.src.api_osmr import get_matrix, parse_coordinates
from app.src.routing import optimize_routes, plot_route

from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import calinski_harabasz_score

import pandas as pd
import numpy as np

from typing import List
from sklearn.cluster import MiniBatchKMeans
from scipy.spatial.distance import cdist

from app.utils.soft_colors import SOFT_COLORS
SOFT_COLORS = [
    "orange",
    "teal",
    "cyan",
    "indigo",
    "lightblue",
    "darkblue", 
    "darkpurple",  
    "gray",       
    "blue" 
]


class ModelHandler(AbstractBaseClass):
    def __init__(self):
        pass

    def load_data(self, path):
        self.input_data = pd.read_csv(path)
        self.X = self.input_data[["Latitud", "Longitud"]].values
        self.soft_colors = SOFT_COLORS


    def find_max_diff(self, input_dictionary: dict, diff_value=1) -> int:
        """
        Finds the number of clusters (k) that maximizes the difference between 
        consecutive Calinski-Harabasz scores.

        Parameters:
        input_dictionary (dict): Keys are cluster numbers (k), values are CH scores.

        Returns:
        int: Optimal number of clusters (k) based on the maximum difference.
        """
        
        sorted_k = sorted(input_dictionary.keys())

        differences = {k: input_dictionary[k] - input_dictionary[sorted_k[i - 1]]
                    for i, k in enumerate(sorted_k[1:], start=1)}

        return max(differences, key=differences.get) - diff_value

    
    def find_best_n_clusters(self, linked, data, min_clusters=20, max_clusters=30, diff_value=1) -> int:
        """Find the best number of clusters based on the max difference between the calinski-harabasz scores"""

        scores_dict = {}

        for k in range(min_clusters, max_clusters):  
            labels = fcluster(linked, k, criterion='maxclust') 
            scores_dict[k] = calinski_harabasz_score(data, labels)

        return self.find_max_diff(scores_dict, diff_value=diff_value)


    def compute_cluster_centroids(self, data, labels):
        unique_clusters = np.unique(labels)
        centroids = []

        for cluster in unique_clusters:
            cluster_points = data[labels == cluster] 
            centroid = cluster_points.mean(axis=0)  
            centroids.append(centroid)

        return np.array(centroids)



        
        
    def balanced_kmeans_labels(self, df, num_routes=3, num_stations_per_route=9):
        """
        Cluster bus stations into balanced groups with equal number of points.

        Args:
            df (DataFrame): DataFrame with columns ['lat', 'lng'].
            num_routes (int): Number of bus routes.
            num_stations_per_route (int): Number of bus stations per route.

        Returns:
            List[int]: List of cluster labels (0, 1, 2) corresponding to input data order.
        """

        
        kmeans = MiniBatchKMeans(n_clusters=num_routes, random_state=42, n_init=10)
        labels = kmeans.fit_predict(df[["lat", "lng"]])

        # Step 2: Balance the Clusters
        df_temp = df.copy()
        df_temp["route_id"] = labels

        max_iterations = 100
        iteration = 0
        while iteration < max_iterations:
            route_counts = df_temp["route_id"].value_counts().to_dict()
            if all(count == num_stations_per_route for count in route_counts.values()):
                break
            iteration += 1

            
            oversized_clusters = {r: c for r, c in route_counts.items() if c > num_stations_per_route}
            undersized_clusters = {r: c for r, c in route_counts.items() if c < num_stations_per_route}

            
            for over_cluster, over_count in oversized_clusters.items():
                excess_points = over_count - num_stations_per_route
                over_points = df_temp[df_temp["route_id"] == over_cluster][["lat", "lng"]].values

                
                for under_cluster, under_count in undersized_clusters.items():
                    under_centroid = df_temp[df_temp["route_id"] == under_cluster][["lat", "lng"]].mean().values.reshape(1, -1)
                    distances = cdist(over_points, under_centroid)

                    
                    move_indices = df_temp[df_temp["route_id"] == over_cluster].index[np.argsort(distances.flatten())[:excess_points]]
                    df_temp.loc[move_indices, "route_id"] = under_cluster

        return df_temp["route_id"].tolist()
    

    def generate_central_station(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Adds a central bus station (mean of all latitude and longitude) to the dataframe.

        Args:
            df (pd.DataFrame): DataFrame with 'lat' and 'lng' columns.

        Returns:
            pd.DataFrame: Updated DataFrame with an additional central station.
        """
        tmp_lng = np.mean(df["lng"])
        tmp_lat = np.mean(df["lat"])
        tmp_df = pd.DataFrame([{"bus_stop":-1, "lat": tmp_lat, "lng":tmp_lng, "route_id":-1}])
        output_df = pd.concat([tmp_df, df]) 
        return output_df
    

    def create_k_dataframes(self, original_df:pd.DataFrame) -> List:
        route_ids = original_df["route_id"].unique().tolist()

        df_list = [original_df[(original_df["route_id"] == route_id) | (original_df["route_id"] == -1)] for route_id in route_ids if route_id >= 0]

        return df_list
    
    
    def ensure_index_starts_at_zero(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Ensures that the DataFrame index starts at 0. If 0 is missing, 
        renumbers the index so the first row gets index 0.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
        
        Returns:
            pd.DataFrame: DataFrame with adjusted index.
        """
        if 0 not in df.index:
            df = df.reset_index(drop=True) 
        return df


    def create_input_df(self):

        route_labels = fcluster(self.linked, self.num_routes, criterion='maxclust')
        station_labels = fcluster(self.linked, self.num_stations, criterion='maxclust')


        station_centroids = self.compute_cluster_centroids(self.X, station_labels)
        route_centroids = self.compute_cluster_centroids(self.X, route_labels)


        self.df = pd.DataFrame(data=station_centroids, columns=["lat", "lng"]).reset_index(names="bus_stop") 
        self.df["bus_stop"] = self.df["bus_stop"].astype(str)  


        route_assignments = []
        for station in station_centroids:
            distances = cdist([station], route_centroids) 
            closest_route = np.argmin(distances)  
            route_assignments.append(closest_route)

        self.df["route_id"] = route_assignments 

    
    def optimization_wrapper(self, df: pd.DataFrame):
        df = self.ensure_index_starts_at_zero(df)
        bus_stops_list = df.bus_stop
        coordinates = parse_coordinates(df)

        response_distances = get_matrix(coordinates)

        distances = np.array(response_distances["distances"]).astype(int)
        starting_stop = df["bus_stop"].iloc[0]

        depot_index = int(bus_stops_list.index[bus_stops_list == starting_stop][0])
        return optimize_routes(distances, depot_index, list(bus_stops_list))



    def map_wrapper(self, df: pd.DataFrame, idx_route, input_color):

        df_best_route = df.iloc[idx_route[0]]

        coordinates = list(zip(df_best_route["lat"], df_best_route["lng"]))

        return plot_route(coordinates, dist=30000, zoom_start=11, input_color=input_color)


    def fit(self, path, num_stations = None, balance_stations=True, generate_central_station=True, num_routes=None, method="ward"):

        # HCA
        self.num_stations  = num_stations
        self.num_routes = num_routes
        self.load_data(path)
        self.linked = sch.linkage(self.X, method=method)
        
        # The first clustering is to find group the POI into a a single cluster
        if self.num_stations is None:
            self.num_stations = self.find_best_n_clusters(linked=self.linked, data=self.X)
        logger.info(f"Num of bus stations: {self.num_stations}") 


        # Cut the dendogram
        # The second is to find the optimal number of routes
        if self.num_routes is None:
            self.num_routes = self.find_best_n_clusters(self.linked, self.X, min_clusters=2, max_clusters=6, diff_value=0)
        
        else:
            logger.info(f"Setting number of routes: {self.num_routes}") 

        # We set the input df 
        self.create_input_df() 


        # Bug 1 -> Non-existing route id
        if balance_stations==True:
            self.df["route_id"] = self.balanced_kmeans_labels(self.df) 
            logger.info("generating labels with kmeans to balance the number of stations")
        
        else:

            logger.info("Not balancing number of stations")


        # Bug 2
        if generate_central_station == True:
            self.df = self.generate_central_station(self.df)
            logger.info("generating central station")

        else:
            logger.info("Not generating central station")
        
        
        self.k_df_list = self.create_k_dataframes(self.df)



    def predict(self, path, num_stations=None, balance_stations=True, generate_central_station=True, num_routes=None, method="ward"):
        logging.info("Starting predict function")
        self.fit(path=path, num_stations=num_stations, balance_stations=balance_stations, generate_central_station=generate_central_station, num_routes=num_routes, method=method)

        optimized_routes = [self.optimization_wrapper(route) for route in self.k_df_list]
        logging.info("Finished optimization_wrapper")

        optimized_maps = [self.map_wrapper(self.k_df_list[i], optimized_routes[i], self.soft_colors[i]) for i in range(len(self.k_df_list))]
        logging.info("Finished map_wrapper")

        return optimized_routes, optimized_maps




PATH = r"C:\JupyterNotebooks\projects\bus-route-builder\app\data\equipaments_model_2.csv"

# handler = ModelHandler()
# a, b = handler.predict(PATH,
#                        num_stations=20,
#                        num_routes=2, 
#                        balance_stations=False,
#                        generate_central_station=False, 
#                         )



load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\Vladi\AppData\Local\pypoetry\Cache\virtualenvs\bus-route-builder-i2MxGmv1-py3.10\lib\site-packages\ortools\.libs\ortools.dll...


In [7]:
from pydantic import BaseModel

class RouteResponse(BaseModel):
    name: str
    input_path: str
    output_path: str
    num_stations: Optional[int] = None 
    num_routes: Optional[int] = None 
    central_station: Optional[bool] = None 
    balanced_num_stops: Optional[bool] = None 
    results: List

    model_config = {
        "response": {
            "name": "Viladecans",
            "input_path": "data/equipaments_model.csv",
            "output_path": "output_folder/image.jpg",
            "num_stations": 30,
            "num_routes": 3,
            "central_station": True,
            "balanced_num_stops": True,
            "results": [{"route_id": 2,
           "route_num_stations": 3,
           "distance": 10.23
           }]
        }
    }

class MapResponse(BaseModel):

    route_id: int
    route_num_stations: int
    distance: float

    model_config = {"route_id": 2,
           "route_num_stations": 3,
           "distance": 10.23
           }
    

class ModelAdapter(ModelHandler):
    def __init__(self):
        self.model_handler = ModelHandler()
        self.random_state = 42
    
    def merge_folium_maps(self, maps: List[folium.Map]) -> folium.Map:
        """
        Merges multiple Folium maps into a single map.

        Args:
            maps (List[folium.Map]): List of Folium map objects.

        Returns:
            folium.Map: A single merged Folium map.
        """

        # Step 1: Create a new base map (use the location of the first map)
        base_map = folium.Map(location=maps[0].location, zoom_start=15, tiles="cartodbpositron")

        # Step 2: Add all layers from each map
        for map_obj in maps:
            for child in map_obj._children.values():
                base_map.add_child(child)

        return base_map
    
    def parse_output(self, output_routes, path, balance_stations, generate_central_station, method="ward"):
        res = [MapResponse.model_validate({"route_id":idx, "route_num_stations": len(output_routes[idx][0]), "distance": output_routes[idx][-1]}) for idx in range(len(output_routes))] 

        return RouteResponse.model_validate({
            "name": "some_name",
            "input_path": path,
            "output_path": "output_dir/",
            "num_stations": self.model_handler.num_stations,
            "num_routes": self.model_handler.num_routes,
            "central_station": generate_central_station,
            "balanced_num_stops": balance_stations,
            "results": res
        })


    def predict(self, path, num_stations=None, balance_stations=True, generate_central_station=True, num_routes=None, method="ward"):
    
        routes, maps = self.model_handler.predict(path, num_stations=num_stations, balance_stations=balance_stations, generate_central_station=generate_central_station, num_routes=num_routes, method=method)
        final_map = self.merge_folium_maps(maps)

        return self.parse_output(routes, path, balance_stations, generate_central_station), final_map
        # return routes, final_map

In [ ]:
# adapter.model_handler.balanced_kmeans_labels()

TypeError: ModelHandler.balanced_kmeans_labels() missing 1 required positional argument: 'df'

In [8]:
from loguru import logger

adapter = ModelAdapter()

a, b = adapter.predict(PATH)

2025-03-09 08:08:02.622 | INFO     | __main__:fit:295 - Num of bus stations: 27
2025-03-09 08:08:02.909 | INFO     | __main__:fit:313 - generating labels with kmeans to balance the number of stations
2025-03-09 08:08:02.911 | INFO     | __main__:fit:323 - generating central station


In [10]:
b

In [ ]:
import io
from PIL import Image


def map_creator(input_map: folium.Map, name: str):
    """
    Saves the map as an image after adjusting the zoom level dynamically 
    to fit all clusters.

    Args:
        input_map (folium.Map): The map object.
        name (str): The name of the output image file.
    """

    marker_coords = []
    for child in input_map._children.values():
        if isinstance(child, folium.Marker):
            marker_coords.append(child.location)
    
    if marker_coords:
        input_map.fit_bounds(marker_coords)

    img_data = input_map._to_png(1) 
    img = Image.open(io.BytesIO(img_data))
    img.save(f'output_folder/{name}_bus_map.png')

    logging.info(f"Map saved for {name} with adjusted bounds.")


# Example usage
map_creator(b, "some_name")


In [1]:
# import pandas as pd
# import pandas as pd

# file_path = r"C:\JupyterNotebooks\projects\bus-route-builder\app\data\equipaments_model_2.csv"
# df = pd.read_csv(file_path, encoding="ISO-8859-1", delimiter=";")

# # Convert latitude and longitude to proper numerical format
# df["Latitud"] = df["Latitud"].str.replace(",", ".").astype(float)
# df["Longitud"] = df["Longitud"].str.replace(",", ".").astype(float)




In [2]:
# pd.read_csv(r"C:\JupyterNotebooks\projects\bus-route-builder\app\data\equipaments_model_2.csv")

In [3]:
# var = True
# int(var)

In [ ]:
# df.to_csv(r"C:\JupyterNotebooks\projects\bus-route-builder\app\data\equipaments_model_2.csv", index=False)

In [4]:
# a

In [16]:
adapter.model_handler.k_df_list[0].to_csv("results/results_df_1.csv")
adapter.model_handler.k_df_list[1].to_csv("results/results_df_2.csv")
adapter.model_handler.k_df_list[2].to_csv("results/results_df_3.csv")

In [20]:
class RouteResponse(BaseModel):
    name: str
    input_path: str
    output_path: str
    num_stations: int
    num_routes: int
    central_station: bool
    balanced_num_stops: bool
    results: List

    model_config = {
        "response": {
            "name": "Viladecans",
            "input_path": "data/equipaments_model.csv",
            "output_path": "output_folder/image.jpg",
            "num_stations": 30,
            "num_routes": 3,
            "central_station": True,
            "balanced_num_stops": True,
            "results": [{"route_id": 2,
           "route_num_stations": 3,
           "distance": 10.23
           }]
        }
    }


res = [MapResponse.model_validate({"route_id":idx, "route_num_stations": len(a[idx][0]), "distance": a[idx][-1]}) for idx in range(len(a))] 
res

RouteResponse.model_validate({
            "name": "some_name",
            "input_path": PATH,
            "output_path": "output_dir",
            "num_stations": int(3),
            "num_routes": int(3),
            "central_station": bool(True),
            "balanced_num_stops": bool(True),
            "results": res
        })

RouteResponse(name='some_name', input_path='C:\\JupyterNotebooks\\projects\\bus-route-builder\\app\\data\\equipaments_model.csv', output_path='output_dir', num_stations=3, num_routes=3, central_station=True, balanced_num_stops=True, results=[MapResponse(route_id=0, route_num_stations=11, distance=10.52), MapResponse(route_id=1, route_num_stations=11, distance=30.89), MapResponse(route_id=2, route_num_stations=11, distance=8.87)])

In [18]:
# [df for df in adapter.handler.k_df_list]

In [19]:
a

[([0, 8, 5, 4, 9, 3, 6, 1, 2, 7, 0], 10.52),
 ([0, 7, 3, 2, 1, 8, 9, 6, 4, 5, 0], 30.89),
 ([0, 8, 9, 4, 2, 3, 6, 1, 5, 7, 0], 8.87)]

In [49]:
RouteResponse.model_validate({
            "name": PATH.split("/")[-1].split(".")[0],
            "input_path": PATH,
            "output_path": "output_dir",
            "num_stations": int(3),
            "num_routes": int(3),
            "central_station": bool(True),
            "balanced_num_stops": bool(True),
            "results": res
        })

RouteResponse(name='C:\\JupyterNotebooks\\projects\\bus-route-builder\\app\\data\\equipaments_model', input_path='C:\\JupyterNotebooks\\projects\\bus-route-builder\\app\\data\\equipaments_model.csv', output_path='output_dir', num_stations=3, num_routes=3, central_station=True, balanced_num_stops=True, results=[MapResponse(route_id=0, route_num_stations=11, distance=10.52), MapResponse(route_id=1, route_num_stations=11, distance=30.89), MapResponse(route_id=2, route_num_stations=11, distance=8.87)])

In [ ]:
idx = 0

[]

map_results = {"route_id": idx,
           "route_num_stations": len(a[idx][0]),
           "distance": a[idx][-1]
           }

In [22]:
results

{'route_id': 0, 'route_num_stations': 2, 'distance': 10.52}

In [1]:
b

NameError: name 'b' is not defined

In [14]:
a

[([0, 8, 5, 4, 9, 3, 6, 1, 2, 7, 0], 10.52),
 ([0, 7, 3, 2, 1, 8, 9, 6, 4, 5, 0], 30.89),
 ([0, 8, 9, 4, 2, 3, 6, 1, 5, 7, 0], 8.87)]

In [ ]:
# optimized_routes = handler.optimization_wrapper(b[0])

In [ ]:
# bus_stops_list = df.bus_stop
# coordinates = parse_coordinates(df)

# response_distances = get_matrix(coordinates)

# distances = np.array(response_distances["distances"]).astype(int)
# starting_stop = df["bus_stop"].iloc[0]

# depot_index = int(bus_stops_list.index[bus_stops_list == starting_stop][0])
# result = optimize_routes(distances, depot_index, list(bus_stops_list))

: 

In [61]:
c = merge_folium_maps(b)
c

In [4]:
def merge_folium_maps(maps: List[folium.Map]) -> folium.Map:
        """
        Merges multiple Folium maps into a single map.

        Args:
            maps (List[folium.Map]): List of Folium map objects.

        Returns:
            folium.Map: A single merged Folium map.
        """

        # Step 1: Create a new base map (use the location of the first map)
        base_map = folium.Map(location=maps[0].location, zoom_start=15, tiles="cartodbpositron")

        # Step 2: Add all layers from each map
        for map_obj in maps:
            for child in map_obj._children.values():
                base_map.add_child(child)

        return base_map